In [24]:
import numpy as np

from scipy.integrate import solve_ivp
from scipy.optimize import root

from ipywidgets import interact, IntSlider, Dropdown

from bokeh.io import push_notebook, show, output_notebook
from bokeh.plotting import figure
from bokeh.layouts import row, column
from bokeh.models import PrintfTickFormatter
output_notebook(hide_banner=True)

# Brusselator model

The dynamics of the oscillating reaction discovered by Belousov and Zhabotinsky, 
can be modeled through the so-called Brusselator model depending on two parameters:

$$
\left\{\begin{aligned}
\mathrm{d}_t y_1 & = a - (b+1) y_1 + y_1^2y_2\\
\mathrm{d}_t y_2 & = b y_1 - y_1^2y_2
\end{aligned}\right.
$$

In [25]:
class brusselator_model:

    def __init__(self, a, b) :
        self.a = a
        self.b = b
    
    def fcn(self, t, y):
        y1, y2 = y
        a = self.a
        b = self.b 
        y1_dot = a - (b+1)*y1 + y1*y1*y2 
        y2_dot = b*y1 - y1*y1*y2  
        return np.array([y1_dot, y2_dot])

    def jac(self, t, y):
        y1, y2 = y
        a = self.a
        b = self.b
        return np.array([[-(b+1)+2*y1*y2 , y1*y1], [b-2*y1*y2, -y1*y1]])

## Quasi-exact solution

The quasi-exact solution is obtained by using an explicit Runge-Kutta method of order 5 with stepsize control and fine tolerances due to Dormand and Prince.

In [26]:
tini = 0. 
tend = 40.

yini = (1.5, 3)

bm = brusselator_model(a=1, b=4)
fcn = bm.fcn  

sol = solve_ivp(fcn, (tini, tend), yini, method="RK45", rtol=1e-9, atol=1e-12)

fig_sol = figure(x_range=(tini, tend), width=800, height=300, title="Solution")
fig_sol.line(sol.t, sol.y[0], legend_label="y1", line_width=2)    
fig_sol.line(sol.t, sol.y[1], legend_label="y2", line_width=2, color="green")
fig_sol.legend.location = "top_left"
fig_sol.xaxis.axis_label = "t"
fig_sol.yaxis.axis_label = "yi"
show(fig_sol)

fig_pp = figure(x_range=(-1, 10), y_range=(-1, 10), width=300, height=300, title="Phase portrait")
fig_pp.line(sol.y[0], sol.y[1], line_width=2)   
fig_pp.xaxis.axis_label = "y1"
fig_pp.yaxis.axis_label = "y2"
show(fig_pp)

## Characterisation of stiffness

In [27]:
tini = 0. 
tend = 40.

yini = (1.5, 3)

bm = brusselator_model(a=1, b=4)
fcn = bm.fcn  
jac = bm.jac

sol = solve_ivp(fcn, (tini, tend), yini, method="RK45", rtol=1e-9, atol=1e-12)

eig_vals = np.zeros((sol.t.size, 2), dtype=np.complex_)
for it in range(0,sol.t.size):
    eig_vals[it] = np.linalg.eigvals(jac(0, sol.y[:,it]))
lambda1 = eig_vals[:, 0]
lambda2 = eig_vals[:, 1]

fig_real = figure(x_range=(tini, tend), plot_height=300, plot_width=800, 
                  title = "Real part of eigenvalues (click on legend entries to hide corresponding plot)")
fig_imag = figure(x_range=(tini, tend), plot_height=300, plot_width=800, 
                  title = "Imaginary part of eigenvalues (click on lengend to hide corresponding plot)")

fig_real.line(sol.t, np.real(lambda1), line_width=2, legend_label="lambda1")
fig_real.line(sol.t, np.real(lambda2), line_width=2, color="green", legend_label="lambda2")
fig_real.legend.click_policy="hide"
fig_real.xaxis.axis_label = "t"

fig_imag.line(sol.t, np.imag(lambda1), line_width=2, legend_label="lambda1")
fig_imag.line(sol.t, np.imag(lambda2), line_width=2, color="green", legend_label="lambda2")
fig_imag.legend.click_policy="hide"
fig_imag.xaxis.axis_label = "t"

show(column(fig_real, fig_imag))

## Runge-Kutta methods

In [28]:
class ode_result:
    def __init__(self, y, t, nfev):
        self.y = y
        self.t = t
        self.nfev = nfev 

def forward_euler(tini, tend, nt, yini, fcn):

    dt = (tend-tini) / (nt-1)
    t = np.linspace(tini, tend, nt)

    yini_array = np.array(yini)
    neq = yini_array.size

    y = np.zeros((neq, nt), order='F')
    y[:,0] = yini_array

    for it, tn  in enumerate(t[:-1]):
        yn = y[:,it]
        y[:,it+1] = yn + dt*fcn(tn, yn)

    nfev = nt-1

    return ode_result(y, t, nfev)

def rk2(tini, tend, nt, yini, fcn):

    dt = (tend-tini) / (nt-1)
    t = np.linspace(tini, tend, nt)

    yini_array = np.array(yini)
    neq = yini_array.size

    y = np.zeros((neq, nt), order='F')
    y[:,0] = yini_array

    for it, tn  in enumerate(t[:-1]):
        yn = y[:,it]
        k1 = fcn(tn, yn)
        k2 = fcn(tn + 0.5*dt, yn + dt*(0.5*k1))
        y[:,it+1] = yn + dt*k2

    nfev = 2*(nt-1)

    return ode_result(y, t, nfev)

def rk3(tini, tend, nt, yini, fcn):

    dt = (tend-tini) / (nt-1)
    t = np.linspace(tini, tend, nt)

    yini_array = np.array(yini)
    neq = yini_array.size

    y = np.zeros((neq, nt), order='F')
    y[:,0] = yini_array

    for it, tn  in enumerate(t[:-1]):
        yn = y[:,it]
        k1 = fcn(tn, yn)
        k2 = fcn(tn + 0.5*dt, yn + dt*(0.5*k1))
        k3 = fcn(tn + dt, yn + dt*(-k1 + 2*k2))
        y[:,it+1] = yn + (dt/6)*(k1+4*k2+k3)

    nfev = 3*(nt-1)

    return ode_result(y, t, nfev)

def rk4(tini, tend, nt, yini, fcn):

    dt = (tend-tini) / (nt-1)
    t = np.linspace(tini, tend, nt)

    yini_array = np.array(yini)
    neq = yini_array.size

    y = np.zeros((neq, nt), order='F')
    y[:,0] = yini_array

    for it, tn  in enumerate(t[:-1]):
        yn = y[:,it]
        k1 = fcn(tn, yn)
        k2 = fcn(tn + 0.5*dt, yn + dt*(0.5*k1))
        k3 = fcn(tn + 0.5*dt, yn + dt*(0.5*k2))
        k4 = fcn(tn + dt, yn + dt*k3)
        y[:,it+1] = yn + (dt/6)*(k1+2*k2+2*k3+k4)

    nfev = 4*(nt-1)

    return ode_result(y, t, nfev)

In [29]:
tini = 0.
tend = 40.

yini = (1.5 , 3)

bm = brusselator_model(a=1, b=4)
fcn = bm.fcn  

nt = 1000
sol_rk = forward_euler(tini, tend, nt, yini, fcn)

sol_exa = solve_ivp(fcn, (tini, tend), yini, method="RK45", rtol=1e-9, atol=1e-12, t_eval=sol_rk.t)
sol_exa_plot = solve_ivp(fcn, (tini, tend), yini, method="RK45", rtol=1e-12, atol=1e-12)
y1_err = np.abs(sol_exa.y[0] - sol_rk.y[0])
y2_err = np.abs(sol_exa.y[1] - sol_rk.y[1])

# plot exact and rk solutions 
fig_sol = figure(x_range=(tini, tend), plot_height=300, plot_width=600, title="Solution")
plt_sol_y1 = fig_sol.x(sol_rk.t, sol_rk.y[0], line_width=2, legend_label="y1")
plt_exa_y1 = fig_sol.line(sol_exa_plot.t, sol_exa_plot.y[0], color="grey")
plt_sol_y2 = fig_sol.x(sol_rk.t, sol_rk.y[0], line_width=2, color="green", legend_label="y2")
plt_exa_y1 = fig_sol.line(sol_exa_plot.t, sol_exa_plot.y[1], color="grey")
fig_sol.xaxis.axis_label = "t"
fig_sol.yaxis.axis_label = "yi"
fig_sol.legend.location = "top_left"

# plot global error
fig_err = figure(x_range=(tini, tend), y_axis_type="log", plot_height=300, plot_width=600, title="Global error")
fig_err.yaxis[0].formatter = PrintfTickFormatter(format="%8.1e")
plt_err_y1 = fig_err.x(sol_rk.t, y1_err, line_width=2, legend_label="y1")
plt_err_y2 = fig_err.x(sol_rk.t, y2_err, line_width=2, color="green", legend_label="y2")
fig_err.xaxis.axis_label = "t"
fig_err.yaxis.axis_label = "|yi-yexa|"
fig_err.legend.location = "top_left"

# plot exact and rk solutions in phase space
fig_pp = figure(x_range=(-1, 8), y_range=(-1, 8), width=300, height=300, title="Phase portrait")
plt_pp_exa = fig_pp.line(sol_exa_plot.y[0], sol_exa_plot.y[1], color="Grey")   
plt_pp_rk = fig_pp.line(sol_rk.y[0], sol_rk.y[1], line_width=2)   
fig_pp.xaxis.axis_label = "y1"
fig_pp.yaxis.axis_label = "y2"

show(row(column(fig_sol, fig_err), fig_pp), notebook_handle=True)

def update(method, nt):
    t = np.linspace(tini, tend, nt)
    if method == "rk1":
        sol_rk = forward_euler(tini, tend, nt, yini, fcn)
    elif method == "rk2":
        sol_rk = rk2(tini, tend, nt, yini, fcn)
    elif method == "rk3":
        sol_rk = rk3(tini, tend, nt, yini, fcn)
    elif method == "rk4":
        sol_rk = rk4(tini, tend, nt, yini, fcn)        

    sol_exa = solve_ivp(fcn, (tini, tend), yini, method="RK45", rtol=1e-9, atol=1e-12, t_eval=sol_rk.t)
    y1_err = np.abs(sol_exa.y[0] - sol_rk.y[0])
    y2_err = np.abs(sol_exa.y[1] - sol_rk.y[1])

    plt_sol_y1.data_source.data = dict(x=sol_rk.t, y=sol_rk.y[0])
    plt_sol_y2.data_source.data = dict(x=sol_rk.t, y=sol_rk.y[1])
    plt_err_y1.data_source.data = dict(x=sol_rk.t, y=y1_err)
    plt_err_y2.data_source.data = dict(x=sol_rk.t, y=y2_err)
    plt_pp_rk.data_source.data = dict(x=sol_rk.y[0], y=sol_rk.y[1])
    push_notebook()

interact(update, 
         nt=IntSlider(min=500, max=5000, value=nt, step=10, continuous_update=False),
         method=Dropdown(options=["rk1", "rk2", "rk3", "rk4"], value='rk1', description='Method:'));

interactive(children=(Dropdown(description='Method:', options=('rk1', 'rk2', 'rk3', 'rk4'), value='rk1'), IntS…

### Computational cost

In [30]:
tini = 0.
tend = 40.

yini = (1.5 , 3)

bm = brusselator_model(a=1, b=4)
fcn = bm.fcn  

nt = [1001, 2001, 5001, 10001, 100001]
fe_rk1=[]
norm_rk1=[]
fe_rk2=[]
norm_rk2=[]
fe_rk3=[]
norm_rk3=[]
fe_rk4=[]
norm_rk4=[]

for nt_i in nt:

    t = np.linspace(tini,tend,nt_i)
    sol_exa = solve_ivp(fcn, (tini, tend), yini, method="RK45", rtol=1.e-12, atol=1.e-12, t_eval=t)

    sol_rk1 = forward_euler(tini, tend, nt_i, yini, fcn)
    fe_rk1.append(sol_rk1.nfev)
    norm_rk1.append(np.linalg.norm(sol_exa.y-sol_rk1.y) / np.sqrt(nt_i))

    sol_rk2 = rk2(tini, tend, nt_i, yini, fcn)
    fe_rk2.append(sol_rk2.nfev)
    norm_rk2.append(np.linalg.norm(sol_exa.y-sol_rk2.y) / np.sqrt(nt_i))

    sol_rk3 = rk3(tini, tend, nt_i, yini, fcn)
    fe_rk3.append(sol_rk3.nfev)
    norm_rk3.append(np.linalg.norm(sol_exa.y-sol_rk3.y) / np.sqrt(nt_i))

    if (nt_i == 100001):
        nti_rk4 = 20001
        sol_exa = solve_ivp(fcn, (tini, tend), yini, method="RK45", 
                            rtol=1.e-12, atol=1.e-12, t_eval=np.linspace(tini,tend,nti_rk4))
        sol_rk4 = rk4(tini, tend, nti_rk4, yini, fcn)
        fe_rk4.append(4*(nti_rk4-1))
        norm_rk4.append(np.linalg.norm(sol_exa.y-sol_rk4.y) / np.sqrt(nti_rk4))
    else:
        sol_rk4 = rk4(tini, tend, nt_i, yini, fcn)
        fe_rk4.append(sol_rk4.nfev)
        norm_rk4.append(np.linalg.norm(sol_exa.y-sol_rk4.y) / np.sqrt(nt_i))


fig = figure(x_axis_type="log", y_axis_type="log", plot_height=400, plot_width=800,
             title= "Norm of global error vs number of function evaluations" )

fig.x(fe_rk1, norm_rk1, legend_label="rk1")
fig.line(fe_rk1, norm_rk1, legend_label="rk1")
fig.x(fe_rk2, norm_rk2, legend_label="rk2", color="green")
fig.line(fe_rk2, norm_rk2, legend_label="rk2", color="green")
fig.x( fe_rk3, norm_rk3, legend_label="rk3", color="red")
fig.line(fe_rk3, norm_rk3, legend_label="rk3", color="red")
fig.x(fe_rk4, norm_rk4, legend_label="rk4", color="indigo")
fig.line(fe_rk4, norm_rk4, legend_label="rk4", color="indigo")
fig.xaxis.axis_label = "Number of function evaluations"
fig.yaxis.axis_label = "Error"

show(fig)

## Implicit Euler

In [31]:
def backward_euler(tini, tend, nt, yini, fcn): 
 
    dt = (tend-tini) / (nt-1)
    t = np.linspace(tini, tend, nt)

    yini_array = np.array(yini)
    neq = yini_array.size

    y = np.zeros((neq, nt), order='F')
    y[:,0] = yini_array
 
    def g(uip1, *args): 
        uip, tip1 = args 
        return uip1 - uip - dt*fcn(tip1, uip1)
 
    for it, tn  in enumerate(t[:-1]):
        yn = y[:,it]
        y0 = yn + dt*fcn(tn, yn)
        # solve y[:,it+1] - y[:,it] - dt * fcn(tini + (it+1)*dt, y[:,it+1]) = 0 
        sol = root(g, y0, (yn, tn+dt))
        y[:,it+1] = sol.x

    nfev = nt-1
         
    return ode_result(y, t, nfev)

In [32]:
tini = 0.
tend = 40.

yini = (1.5 , 3)

bm = brusselator_model(a=1, b=4)
fcn = bm.fcn  

nt = 1000
sol_rk = backward_euler(tini, tend, nt, yini, fcn)

sol_exa = solve_ivp(fcn, (tini, tend), yini, method="RK45", rtol=1e-12, atol=1e-12, t_eval=sol_rk.t)
sol_exa_plot = solve_ivp(fcn, (tini, tend), yini, method="RK45", rtol=1e-12, atol=1e-12)
y1_err = np.abs(sol_exa.y[0] - sol_rk.y[0])
y2_err = np.abs(sol_exa.y[1] - sol_rk.y[1])

# plot exact and rk solutions 
fig_sol = figure(x_range=(tini, tend), plot_height=300, plot_width=600, title="Solution")
plt_sol_y1 = fig_sol.x(sol_rk.t, sol_rk.y[0], line_width=2, legend_label="y1")
plt_exa_y1 = fig_sol.line(sol_exa_plot.t, sol_exa_plot.y[0], color="grey")
plt_sol_y2 = fig_sol.x(sol_rk.t, sol_rk.y[0], line_width=2, color="green", legend_label="y2")
plt_exa_y1 = fig_sol.line(sol_exa_plot.t, sol_exa_plot.y[1], color="grey")
fig_sol.xaxis.axis_label = "t"
fig_sol.yaxis.axis_label = "yi"
fig_sol.legend.location = "top_left"

# plot global error
fig_err = figure(x_range=(tini, tend), y_axis_type="log", plot_height=300, plot_width=600, title="Global error")
fig_err.yaxis[0].formatter = PrintfTickFormatter(format="%8.1e")
plt_err_y1 = fig_err.x(sol_rk.t, y1_err, line_width=2, legend_label="y1")
plt_err_y2 = fig_err.x(sol_rk.t, y2_err, line_width=2, color="green", legend_label="y2")
fig_err.xaxis.axis_label = "t"
fig_err.yaxis.axis_label = "|yi-yexa|"
fig_err.legend.location = "top_left"

# plot exact and rk solutions in phase space
fig_pp = figure(x_range=(-1, 8), y_range=(-1, 8), width=300, height=300, title="Phase portrait")
plt_pp_exa = fig_pp.line(sol_exa_plot.y[0], sol_exa_plot.y[1], color="Grey")   
plt_pp_rk = fig_pp.line(sol_rk.y[0], sol_rk.y[1], line_width=2)   
fig_pp.xaxis.axis_label = "y1"
fig_pp.yaxis.axis_label = "y2"

show(row(column(fig_sol, fig_err), fig_pp), notebook_handle=True)

def update(nt):
    t = np.linspace(tini, tend, nt)
    sol_rk = backward_euler(tini, tend, nt, yini, fcn)
    sol_exa = solve_ivp(fcn, (tini, tend), yini, method="RK45", rtol=1e-12, atol=1e-12, t_eval=sol_rk.t)
    y1_err = np.abs(sol_exa.y[0] - sol_rk.y[0])
    y2_err = np.abs(sol_exa.y[1] - sol_rk.y[1])

    plt_sol_y1.data_source.data = dict(x=sol_rk.t, y=sol_rk.y[0])
    plt_sol_y2.data_source.data = dict(x=sol_rk.t, y=sol_rk.y[1])
    plt_err_y1.data_source.data = dict(x=sol_rk.t, y=y1_err)
    plt_err_y2.data_source.data = dict(x=sol_rk.t, y=y2_err)
    plt_pp_rk.data_source.data = dict(x=sol_rk.y[0], y=sol_rk.y[1])
    push_notebook()

interact(update, nt=IntSlider(min=50, max=5000, value=nt, step=10, continuous_update=False));

interactive(children=(IntSlider(value=1000, continuous_update=False, description='nt', max=5000, min=50, step=…

## Enbedded Runge-Kutta methods (based on 3/8 fourth-order)

In [33]:
def rk38(tini, tend, nt, yini, fcn):

    # butcher table of the "3/8 rule" 
    c2 = 1/3
    c3 = 2/3
    c4 = 1

    a21 = 1/3
    a31 = -1/3
    a32 = 1
    a41 = 1
    a42 = -1
    a43 = 1

    b1 = 1/8
    b2 = 3/8
    b3 = 3/8
    b4 = 1/8

    dt = (tend-tini) / (nt-1)
    t = np.linspace(tini, tend, nt)

    yini_array = np.array(yini)
    neq = yini_array.size

    y = np.zeros((neq, nt), order='F')
    y[:,0] = yini_array

    for it, tn  in enumerate(t[:-1]):
        yn = y[:,it]
        k1 = fcn(tn, yn)
        k2 = fcn(tn + c2*dt, yn + dt*(a21*k1))
        k3 = fcn(tn + c3*dt, yn + dt*(a31*k1 + a32*k2))
        k4 = fcn(tn + c4*dt, yn + dt*(a41*k1 + a42*k2 + a43*k3))
        y[:,it+1] = yn + dt*(b1*k1+b2*k2+b3*k3+b4*k4)

    nfev = 4*(nt-1)

    return ode_result(y, t, nfev)

class embedded_ode_result:

    def __init__(self, y, t, nfev, t_rej, dt, dt_rej, loc_err_est, loc_err):
        self.y = y
        self.t = t
        self.nfev = nfev 
        self.t_rej = t_rej
        self.dt = dt
        self.dt_rej = dt_rej
        self.loc_err_est = loc_err_est
        self.loc_err = loc_err
        
def rk_embedded(tini, tend, yini, fcn, tol, dt_ini=1.e-2):

    # butcher table of the "3/8 rule" 
    c2 = 1/3
    c3 = 2/3
    c4 = 1

    a21 = 1/3
    a31 = -1/3
    a32 = 1
    a41 = 1
    a42 = -1
    a43 = 1

    b1 = 1/8
    b2 = 3/8
    b3 = 3/8
    b4 = 1/8

    # coefficients bhat of a third order method 
    b1hat = 2*b1 - 1/6 
    b2hat = 2*(1-c2)*b2 
    b3hat = 2*(1-c3)*b3
    b4hat = 0.
    b5hat = 1/6
     
    dt = dt_ini

    yini_array = np.array(yini)
    neq = yini_array.size

    t_acc = [tini]
    t_rej = []
    dt_acc = []
    dt_rej = []
    y = [yini_array] 
    loc_err_est = [0.]
    loc_err = [0.]

    tn = tini
    yn = yini_array

    k5 = np.array(fcn(tn, yn))
    nfev = 1

    while (tn < tend):

        k1 = k5
        k2 = fcn(tn + c2*dt, yn + dt*a21*k1)
        k3 = fcn(tn + c3*dt, yn + dt*(a31*k1 + a32*k2))
        k4 = fcn(tn + c4*dt, yn + dt*(a41*k1 + a42*k2 + a43*k3))

        ynp1 = yn + dt*(b1*k1 + b2*k2 + b3*k3 + b4*k4)

        k5 = np.array(fcn(tn+dt, ynp1))
        nfev += 4

        yhatnp1 = yn + dt*(b1hat*k1 + b2hat*k2 + b3hat*k3 + b4hat*k4 + b5hat*k5)

        err = np.sqrt(1./neq) * np.linalg.norm((ynp1 - yhatnp1)/(1+np.maximum(np.abs(yn), np.abs(ynp1))))
        dtopt = dt * min(5, max(0.2, 0.9*np.power(tol/err, 1./4.)))

        if (err < tol):

            # compute and save exact local error
            sol = solve_ivp(fcn, (tn, tn+dt), yn, method="RK45", rtol=1.e-12, atol=1.e-12)
            loc_err.append(np.sqrt(1./neq) * np.linalg.norm(ynp1 - sol.y[:,-1]))
            
            # update for next step
            tn = tn + dt
            yn = ynp1

            # save time, solution and estimate local error
            t_acc.append(tn)
            dt_acc.append(dt) 
            y.append(yn)
            loc_err_est.append(err)

            # compute next dt
            dt = min(dtopt, tend-tn)

        else:
            t_rej.append(tn)
            dt_rej.append(dt) 
            dt = dtopt
            k5 = k1

    y = np.array(np.transpose(np.array(y)), order='F')

    return embedded_ode_result(y, np.array(t_acc), nfev,  np.array(t_rej), np.array(dt_acc), np.array(dt_rej),
                               np.array(loc_err_est), np.array(loc_err))

In [34]:
tini = 0. 
tend = 40.

yini = (1.5 , 3)

bm = brusselator_model(a=1, b=4)
fcn = bm.fcn  

tol = 1.e-4
sol_rk_emb = rk_embedded(tini, tend, yini, fcn, tol)

sol_exa = solve_ivp(fcn, (tini, tend), yini, method="RK45", rtol=1.e-12, atol=1.e-12, t_eval=sol_rk_emb.t)
sol_exa_plot = solve_ivp(fcn, (tini, tend), yini, method="RK45", rtol=1.e-12, atol=1.e-12)
glob_err = np.empty(sol_rk_emb.t.size)
for i in range(sol_rk_emb.t.size):
    glob_err[i] = np.sqrt(1./2) * np.linalg.norm(sol_exa.y[:,i] - sol_rk_emb.y[:,i])

# plot exact and rk solutions     
fig_sol = figure(x_range=(tini, tend), width=950, height=300, title="Solution")
fig_sol.line(sol_exa_plot.t, sol_exa_plot.y[0], color="Grey")
fig_sol.line(sol_exa_plot.t, sol_exa_plot.y[1], color="Grey")
plt_y1 = fig_sol.x(sol_rk_emb.t, sol_rk_emb.y[0], legend_label="y1", line_width=2)    
plt_y2 = fig_sol.x(sol_rk_emb.t, sol_rk_emb.y[1], legend_label="y2", line_width=2, color="Green")
fig_sol.legend.location = "top_left"
fig_sol.xaxis.axis_label = "t"

# plot time steps 
fig_dt = figure(x_range=(tini, tend), plot_height=300, plot_width=950, y_axis_type="log", 
                title = "Accepted (black) and rejected (red) times step") 
plt_dt = fig_dt.square(sol_rk_emb.t[0:-1], sol_rk_emb.dt, color="Black")
plt_line_dt = fig_dt.line(sol_rk_emb.t[0:-1], sol_rk_emb.dt, color="Black")
plt_dt_rej = fig_dt.circle_x(sol_rk_emb.t_rej, sol_rk_emb.dt_rej, color="Crimson")
fig_dt.xaxis.axis_label = "t"
fig_dt.yaxis.axis_label = "step size"

# plot erros 
fig_err = figure(x_range=(tini, tend), y_axis_type="log", width=950, height=300, title="Error")
plt_loc_err = fig_err.x(sol_rk_emb.t, sol_rk_emb.loc_err_est, legend_label="estimate local error", line_width=2)
plt_loc_err_exa = fig_err.circle(sol_rk_emb.t, sol_rk_emb.loc_err, legend_label="local error", fill_alpha=0., color="crimson")
plt_glob_err = fig_err.square(sol_rk_emb.t, glob_err, legend_label="global error", fill_alpha=0, color="green")
plt_tol = fig_err.line(sol_rk_emb.t, tol*np.ones(sol_rk_emb.t.size), color="grey", line_dash="dotted")    
fig_err.legend.location = "bottom_left"
fig_err.legend.orientation = "horizontal"
fig_err.xaxis.axis_label = "t"

# plot exact and rk solutions in phase space
fig_pp = figure(x_range=(-1, 8), y_range=(-1, 8), width=300, height=300, title="Phase portrait")
plt_pp_exa = fig_pp.line(sol_exa_plot.y[0], sol_exa_plot.y[1], color="Grey")   
plt_pp_rk_emb = fig_pp.line(sol_rk_emb.y[0], sol_rk_emb.y[1], line_width=2)   
fig_pp.xaxis.axis_label = "y1"
fig_pp.yaxis.axis_label = "y2"

show(column(fig_sol, fig_dt, fig_err, fig_pp), notebook_handle=True)

def update(tol):

    sol_rk_emb = rk_embedded(tini, tend, yini, fcn, tol)

    sol_exa = solve_ivp(fcn, (tini, tend), yini, method="RK45", rtol=1.e-12, atol=1.e-12, t_eval=sol_rk_emb.t)
    glob_err = np.empty(sol_rk_emb.t.size)
    for i in range(sol_rk_emb.t.size):
        glob_err[i] = np.sqrt(1./2) * np.linalg.norm(sol_exa.y[:,i] - sol_rk_emb.y[:,i])


    plt_y1.data_source.data = dict(x=sol_rk_emb.t, y=sol_rk_emb.y[0])
    plt_y2.data_source.data = dict(x=sol_rk_emb.t, y=sol_rk_emb.y[1]) 

    plt_dt.data_source.data = dict(x=sol_rk_emb.t[0:-1], y=sol_rk_emb.dt)
    plt_line_dt.data_source.data = dict(x=sol_rk_emb.t[0:-1], y=sol_rk_emb.dt) 
    plt_dt_rej.data_source.data = dict(x=sol_rk_emb.t_rej, y=sol_rk_emb.dt_rej)

    plt_loc_err.data_source.data = dict(x=sol_rk_emb.t, y=sol_rk_emb.loc_err_est)
    plt_loc_err_exa.data_source.data  = dict(x=sol_rk_emb.t, y=sol_rk_emb.loc_err)
    plt_glob_err.data_source.data = dict(x=sol_rk_emb.t, y=glob_err)
    plt_tol.data_source.data = dict(x=sol_rk_emb.t, y=tol*np.ones(sol_rk_emb.t.size))
    
    plt_pp_rk_emb.data_source.data = dict(x=sol_rk_emb.y[0], y=sol_rk_emb.y[1])

    print("   Number of time step : " + str(sol_rk_emb.t.size-1))
    print("   Number of function evaluations : " + str(sol_rk_emb.nfev))

    push_notebook()


dtol={'1.e-1':1.e-1, '1.e-2':1.e-2, '1.e-3':1.e-3, '1.e-4':1.e-4, '1.e-5':1.e-5, '1.e-6':1.e-6}     
interact(update, tol=Dropdown(options=dtol, value=1.e-4, description='tol'));

interactive(children=(Dropdown(description='tol', index=3, options={'1.e-1': 0.1, '1.e-2': 0.01, '1.e-3': 0.00…

### Computational cost

In [16]:
tini = 0. 
tend = 40.

yini = (1.5 , 3)

bm = brusselator_model(a=1, b=4)
fcn = bm.fcn  

nt = [601, 1001, 2001, 5001, 10001, 20001]
fe_rk38=[]
norm_rk38=[]

for nt_i in nt:
    sol_exa = solve_ivp(fcn, (tini, tend), yini, method="DOP853", rtol=1e-12, atol=1e-12, t_eval=np.linspace(tini, tend, nt_i))
    sol_yrk38 = rk38(tini, tend, nt_i, yini, fcn)
    fe_rk38.append(sol_yrk38.nfev)
    norm_rk38.append(np.linalg.norm(sol_exa.y-sol_yrk38.y) / np.sqrt(nt_i))

tol = [1.e-4, 1.e-6, 1.e-8, 1.e-10, 1e-11]
fe_rk_emb=[]
norm_rk_emb=[]

for tol_i in tol:
    sol_rk_emb = rk_embedded(tini, tend, yini, fcn, tol_i)
    sol_exa = solve_ivp(fcn, (tini, tend), yini, method="DOP853", rtol=1.e-12, atol=1.e-12, t_eval=sol_rk_emb.t)
    fe_rk_emb.append(sol_rk_emb.nfev)
    err = 0 
    for i in range(sol_rk_emb.dt.size):
        ldt = sol_rk_emb.dt[i]
        err += ldt * ((sol_exa.y[0,i+1]-sol_rk_emb.y[0,i+1])*(sol_exa.y[0,i+1]-sol_rk_emb.y[0,i+1]) + \
                      (sol_exa.y[1,i+1]-sol_rk_emb.y[1,i+1])*(sol_exa.y[1,i+1]-sol_rk_emb.y[1,i+1])  )
    err = np.sqrt(err)    
    norm_rk_emb.append(err)

fig = figure(x_axis_type="log", y_axis_type="log", plot_height=500, plot_width=900, \
             title= "Number of function evaluations vs norm of global error" )
fig.x(fe_rk38, norm_rk38, legend_label="rk38", color="indigo")
fig.line(fe_rk38, norm_rk38, legend_label="rk38", color="indigo")
fig.x(fe_rk_emb, norm_rk_emb, legend_label="rk embedded", color="green")
fig.line(fe_rk_emb, norm_rk_emb, legend_label="rk embedded", color="green")
fig.xaxis.ticker = [2500, 5000, 10000, 50000, 100000]
fig.xaxis.axis_label = "Number of function evaluations"
fig.yaxis.axis_label = "Error"

show(fig)